In [15]:
import requests
import json
import hashlib
import time

input = {
    "input": " ",
    "prompt": "问答",
    "question": "以下是一位患者的症状，请帮我分析其疑似诊断可能是哪些病。该患者的症状信息为：问题：小王（男，9岁），最近出现了心率加快、神情极度恐惧、尖叫等症状，并且在夜晚经常惊醒。\n回答",
    "<ans>":"",
}

endpoint_name = "cpm-bee-230915061054KERH"
ak = "9ae2487f531411eeb4eb0242ac120004"
sk = "ISzGfWFBFGd0CEdV=-ovFYV/H0H#RkFj"
host = "saas-1222385307.us-west-2.elb.amazonaws.com"

timestemp = str(int(time.time()))
hl = hashlib.md5()
hl.update(f"{timestemp}{sk}".encode(encoding='utf-8'))
sign = hl.hexdigest()

url = f"http://{host}/inference"
payload = {
   "endpoint_name": endpoint_name,
   "input": json.dumps(input),
   "ak": ak,
   "timestamp": timestemp,
   "sign": sign
}

response = requests.post(url=url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
if response.status_code != 200:
    print(response.reason, response.text)
else:
    obj_json = response.json()
    if obj_json['code'] != 0:
        print(obj_json['<ans>'])
    else:
        data_str = obj_json['data']['data']
        data_obj = json.loads(data_str)
        print(data_obj['<ans>'])

根据小王的症状描述，他可能患有以下几种疾病：
1. 焦虑症：心率加快、神情极度恐惧和尖叫等症状可能是焦虑症的表现。焦虑症是一种常见的心理障碍，常常伴随着失眠、易怒和注意力不集中等症状。
2. 惊恐发作：惊恐发作是一种突然出现的强烈恐惧感和心跳加速的症状。这种症状通常在患者感到害怕或受到惊吓时发生。惊恐发作可以是短暂的，也可以持续数小时甚至更长时间。
3. 睡眠呼吸暂停综合症：睡眠呼吸暂停综合症是一种由于睡眠期间频繁出现上气道阻塞而导致的睡眠障碍。这种疾病常常与心率加快、呼吸急促和白天嗜睡等症状一起出现。


In [2]:
from langchain import SerpAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.tools.python.tool import PythonAstREPLTool

from typing import Dict, Tuple
import os
import json


# 为了使用谷歌搜索（SERPAPI）， WolframAlpha，您需要自行申请它们的 API KEY，然后填入此处
os.environ['SERPAPI_API_KEY'] = '7439b4bb954e332c71776ee046986e08070b6e5db355af2edc1d6072bad17c14'
os.environ['WOLFRAM_ALPHA_APPID'] = '7439b4bb954e332c71776ee046986e08070b6e5db355af2edc1d6072bad17c14'

search = SerpAPIWrapper()
WolframAlpha = WolframAlphaAPIWrapper()
python = PythonAstREPLTool()


def tool_wrapper_for_qwen(tool):
    def tool_(query):
        query = json.loads(query)["query"]
        return tool.run(query)
    return tool_


# 工具描述：
TOOLS = [
    {
        'name_for_human':
            'google search',
        'name_for_model':
            'Search',
        'description_for_model':
            'useful for when you need to answer questions about current events.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "search query of google",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(search)
    },
    {
        'name_for_human':
            'Wolfram Alpha',
        'name_for_model':
            'Math',
        'description_for_model':
            'Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "the problem to solved by Wolfram Alpha",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(WolframAlpha)
    },
    {
        'name_for_human':
            'python',
        'name_for_model':
            'python',
        'description_for_model':
            "A Python shell. Use this to execute python commands. When using this tool, sometimes output is abbreviated - Make sure it does not look abbreviated before using it in your answer. "
            "Don't add comments to your python code.",
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "a valid python command.",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(python)
    }

]


In [4]:
TOOL_DESC = """{name_for_model}: Call this tool to interact with the {name_for_human} API. What is the {name_for_human} API useful for? {description_for_model} Parameters: {parameters} Format the arguments as a JSON object."""

REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tool_descs}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {query}"""


def build_planning_prompt(TOOLS, query):
    tool_descs = []
    tool_names = []
    for info in TOOLS:
        tool_descs.append(
            TOOL_DESC.format(
                name_for_model=info['name_for_model'],
                name_for_human=info['name_for_human'],
                description_for_model=info['description_for_model'],
                parameters=json.dumps(
                    info['parameters'], ensure_ascii=False),
            )
        )
        tool_names.append(info['name_for_model'])
    tool_descs = '\n\n'.join(tool_descs)
    tool_names = ','.join(tool_names)

    prompt = REACT_PROMPT.format(
        tool_descs=tool_descs, tool_names=tool_names, query=query)
    return prompt


prompt_1 = build_planning_prompt(TOOLS[0:1], query="你好,我最近失眠了")
print(prompt_1)


Answer the following questions as best you can. You have access to the following tools:

Search: Call this tool to interact with the google search API. What is the google search API useful for? useful for when you need to answer questions about current events. Parameters: [{"name": "query", "type": "string", "description": "search query of google", "required": true}] Format the arguments as a JSON object.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 你好,我最近失眠了


In [23]:
import requests
import json
import hashlib
import time

input = {
    "input": " 最近失眠啦",
    "prompt": "问答",
    "question": prompt_1,
    "<ans>": "",
}

endpoint_name = "cpm-bee-230915061054KERH"
ak = "9ae2487f531411eeb4eb0242ac120004"
sk = "ISzGfWFBFGd0CEdV=-ovFYV/H0H#RkFj"
host = "saas-1222385307.us-west-2.elb.amazonaws.com"

timestemp = str(int(time.time()))
hl = hashlib.md5()
hl.update(f"{timestemp}{sk}".encode(encoding='utf-8'))
sign = hl.hexdigest()

url = f"http://{host}/inference"

payload = {
    "endpoint_name": endpoint_name,
    "input": json.dumps(input),
    "ak": ak,
    "timestamp": timestemp,
    "sign": sign
}

response = requests.post(url=url, data=json.dumps(payload), headers={
                         'Content-Type': 'application/json'})
if response.status_code != 200:
    print(response.reason, response.text)
else:
    obj_json = response.json()
    if obj_json['code'] != 0:
        print(obj_json['<ans>'])
    else:
        data_str = obj_json['data']['data']
        data_obj = json.loads(data_str)
        print(data_obj['<ans>'])


Thought: 你好，你最近失眠了。为了改善睡眠质量，我有一些建议：
1. 保持规律的作息时间；
2. 创造一个舒适的睡眠环境；
3. 避免在睡觉前过度使用电子设备；
4. 尝试放松身心，如冥想或深呼吸练习等。
Action Input: 无
Observation Result: 通过以上建议，我相信你的睡眠质量会有所提高。


In [6]:
# 从llm的输出中解析需要使用的工具和入参，并调用对应工具
def parse_latest_plugin_call(text: str) -> Tuple[str, str]:
    i = text.rfind('\nAction:')
    j = text.rfind('\nAction Input:')
    k = text.rfind('\nObservation:')
    if 0 <= i < j:  # If the text has `Action` and `Action input`,
        if k < j:  # but does not contain `Observation`,
            # then it is likely that `Observation` is ommited by the LLM,
            # because the output text may have discarded the stop word.
            text = text.rstrip() + '\nObservation:'  # Add it back.
            k = text.rfind('\nObservation:')
            pass
    if 0 <= i < j < k:
        plugin_name = text[i + len('\nAction:'):j].strip()
        plugin_args = text[j + len('\nAction Input:'):k].strip()
        return plugin_name, plugin_args
    return '', ''


def use_api(tools, response):
    use_toolname, action_input = parse_latest_plugin_call(response)
    if use_toolname == "":
        return "no tool founds"

    used_tool_meta = list(
        filter(lambda x: x["name_for_model"] == use_toolname, tools))
    if len(used_tool_meta) == 0:
        return "no tool founds"

    api_output = used_tool_meta[0]["tool_api"](action_input)
    return api_output


In [10]:
data_obj['<ans>']


'Thought: 你好，你最近失眠了吗？\nAction Input: 没有任何信息可以提供。\nObservation: 失眠是一种常见的睡眠障碍，可能会导致疲劳、注意力不集中和情绪波动等问题。建议保持健康的生活方式，包括规律作息时间、均衡饮食以及适量运动。同时，也可以尝试一些放松技巧，如深呼吸、冥想或渐进性肌肉松弛等来缓解失眠症状。\nFinal Answer: 是的，我最近失眠了。为了缓解失眠症状，我应该保持健康的生活方式，包括规律作息时间、均衡饮食以及适量运动。同时，也可以尝试一些放松技巧，如深呼吸、冥想或渐进性肌肉松弛等来缓解失眠症状。'

In [22]:
import requests
import json
import hashlib
import time

input = {
    "input": " ",
    "prompt": "问答",
    "question": "你好,我最近失眠了",
    "<ans>": "",
}

endpoint_name = "cpm-bee-230915061054KERH"
ak = "9ae2487f531411eeb4eb0242ac120004"
sk = "ISzGfWFBFGd0CEdV=-ovFYV/H0H#RkFj"
host = "saas-1222385307.us-west-2.elb.amazonaws.com"

timestemp = str(int(time.time()))
hl = hashlib.md5()
hl.update(f"{timestemp}{sk}".encode(encoding='utf-8'))
sign = hl.hexdigest()

url = f"http://{host}/inference"
payload = {
    "endpoint_name": endpoint_name,
    "input": json.dumps(input),
    "ak": ak,
    "timestamp": timestemp,
    "sign": sign
}

response = requests.post(url=url, data=json.dumps(payload), headers={
                         'Content-Type': 'application/json'})
if response.status_code != 200:
    print(response.reason, response.text)
else:
    obj_json = response.json()
    if obj_json['code'] != 0:
        print(obj_json['<ans>'])
    else:
        data_str = obj_json['data']['data']
        data_obj = json.loads(data_str)
        print(data_obj['<ans>'])


你好，我很抱歉听到你最近失眠了。失眠可能会影响你的情绪、健康和工作效率。以下是一些可能有帮助的建议：

1. 保持规律的作息时间：尽量每天都在同一时间上床睡觉和起床，这有助于调整你的生物钟。

2. 创造一个舒适的睡眠环境：确保你的卧室安静、凉爽、黑暗、通风良好，并使用舒适的床垫和枕头。

3. 避免刺激物和咖啡因：尽量避免饮用含咖啡因的饮料，如咖啡和茶，因为它们会刺激你的神经系统，导致失眠。

4. 放松身心：尝试一些放松技巧，如冥想、深呼吸或渐进性肌肉松弛等，以减轻压力和焦虑感。

5. 定期锻炼：适量的运动可以帮助改善睡眠质量，但要注意不要在睡前过于剧烈地运动。

希望这些建议能对你有所帮助，如果情况严重，请务必寻求医生的帮助。


In [24]:
!git clone https://github.com/modelscope/modelscope-agent.git

Cloning into 'modelscope-agent'...
remote: Enumerating objects: 701, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 701 (delta 152), reused 164 (delta 96), pack-reused 419
Receiving objects: 100% (701/701), 21.17 MiB | 228.00 KiB/s, done.
Resolving deltas: 100% (391/391), done.


In [26]:
!cd modelscope-agent && pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 3.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 9.2 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 5.4 MB/s eta 0:00:0000: